### Vorbereitung Arbeitsumgebung

In [21]:
import arcpy
from arcpy.ia import ExtractBand
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from arcpy.sa import *
import os

In [28]:
 # Setze dein Arbeitsverzeichnis
arcpy.env.workspace = r"Beispieldaten\data" # Pfad an Arbeitsumgebung anpassen


### Raster vorbereiten (Hangrutschungen und Index zusammenfügen)

In [29]:
# Eingaberaster
parameter_raster = "\Weighting_Faktor\SI_sum_weighted.tif"
landslide_raster = "Hangrutschungen.tif"

# Temporärer Pfad für das kombinierte Raster
composite_output = "SI_train_slides.tif"

# Kombiniere Rasterbänder: Band 1 = SI, Band 2 = Rutschung
arcpy.management.CompositeBands([parameter_raster, landslide_raster], composite_output)

<Result 'C:\\Users\\Marlenelokal\\OneDrive\\Dokumente\\UniJena\\Geo403_Geoinfo_Projekt\\Geoinfo_Projekt\\Automatisation\\data\\SI_train_slides.tif'>

### Berechnung von ROC und AUC

In [31]:
composite_raster1 = "SI_train_slides.tif"

# Band 1 (SI) extrahieren
si_raster1 = ExtractBand(composite_raster1, [1])

# Band 2 (Landslide) extrahieren
landslide_raster1 = ExtractBand(composite_raster1, [2])

# Konvertiere in NumPy-Arrays
si_array1 = arcpy.RasterToNumPyArray(si_raster1)
landslide_array1 = arcpy.RasterToNumPyArray(landslide_raster1)

# Maske für gültige Werte (SI hat NoData Wert von -32768)
valid_mask1 = (si_array1 != 3) 
y_true1 = landslide_array1[valid_mask1].flatten()    # 1 = Erdrutsch, 0 = Kein Erdrutsch
y_scores1 = si_array1[valid_mask1].flatten()           # Vorhersagewerte
# Sicherstellen, dass nur 0 und 1 vorkommen (3 war noch NoData Value im binary raster)
y_true1 = (y_true1 == 1).astype(int)

# Berechne ROC-Kurve und AUC
fpr1, tpr1, _ = roc_curve(y_true1, y_scores1)
roc_auc1 = auc(fpr1, tpr1)
print(f"AUC: {roc_auc1}")

AUC: 0.894843711791643


### Darstellung

In [32]:
plt.figure(figsize=(10, 10))

# ROC-Kurven zeichnen
plt.plot(fpr1, tpr1, color="#1f77b4", lw=2, linestyle="-", label="SI (AUC = {:.2f})".format(roc_auc1))

# Diagonale Linie (Random Guess)
plt.plot([0, 1], [0, 1], color="grey", linestyle="--", lw=1)

# Plot-Details
plt.xlabel("Falsch Positiv", fontsize=12)
plt.ylabel("Korrekt Positiv", fontsize=12)
plt.title("ROC-Kurven für Erdrutschmodelle", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle=":", linewidth=0.5)

# Zeige Plot
plt.tight_layout()
plt.show()